# Getting Started

## Data Harmonization with `bdikit`

Data harmonization is the process of integrating and aligning data from different sources into a consistent format to ensure compatibility and interoperability across data analyses and systems. `bdikit` is a library the helps with key data harmonization steps:
- *Schema Mapping*: In this step, data from various sources are mapped to a unified schema or model. This involves identifying equivalent table columns and establishing relationships between disparate datasets.
- *Value Mapping (Data Standardization)*: This step involves converting data into a common format or structure, using consistent naming conventions, units, and coding systems to ensure uniformity.

In this example, we describe how `bdikit` can be used to harmonize datasets from two papers:
- Dou et al. 2020, Proteogenomic Characterization of Endometrial Carcinoma. https://pubmed.ncbi.nlm.nih.gov/32059776/ (CPTAC Discovery)
- Dou et al. 2023, Proteogenomic insights suggest druggable pathways in endometrial carcinoma. https://pubmed.ncbi.nlm.nih.gov/37567170/ (CPTAC Confirmatory)

#### Loading the data

First, import the `bdikit` library and other libraries.

In [1]:
import bdikit as bdi
import pandas as pd

Next, we load our source data using Pandas and select some columns we are interested in.

In [2]:
df_source = pd.read_csv("./datasets/Dou-ucec-discovery.csv")
df_source.head(5)

,idx,Proteomics_Participant_ID,Case_excluded,Proteomics_TMT_batch,Proteomics_TMT_plex,Proteomics_TMT_channel,Proteomics_Parent_Sample_IDs,Proteomics_Aliquot_ID,Proteomics_Tumor_Normal,Proteomics_OCT,...,RNAseq_R1_sample_type,RNAseq_R1_filename,RNAseq_R1_UUID,RNAseq_R2_sample_type,RNAseq_R2_filename,RNAseq_R2_UUID,miRNAseq_sample_type,miRNAseq_UUID,Methylation_available,Methylation_quality
0,S001,C3L-00006,No,2,5,128N,C3L-00006-01,CPT0001460012,Tumor,No,...,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_TAGCTT_S17...,8a1efc47-1c29-417f-a425-cdbd09565dcb,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_TAGCTT_S17...,8c3fe9b7-7acd-4867-8d9c-a8e5d1516eda,Tumor,37bcba98-1094-459e-83ae-c23a602416fb,YES,PASS
1,S002,C3L-00008,No,4,16,130N,C3L-00008-01,CPT0001300009,Tumor,No,...,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_GGCTAC_S22...,555725e8-cba5-4676-9b0a-80100cbf9f47,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_GGCTAC_S22...,15235b12-b67a-4678-acc4-ed03d642bd5e,Tumor,492b50d8-ec35-46e7-a65d-06512aaee394,YES,PASS
2,S003,C3L-00032,No,1,2,131,C3L-00032-01,CPT0001420009,Tumor,No,...,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_GTCCGC_S18...,9ae968f3-691d-4db3-9977-1ab3e5af9085,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_GTCCGC_S18...,423b6b09-02aa-4f47-9241-f75c1dad1161,Tumor,1794ff56-db2d-4d1a-8758-cab7fe3d98c1,YES,PASS
3,S004,C3L-00084,Yes,3,11,129N,C3L-00084-01,CPT0000820012,Tumor,No,...,Tumor,170818_UNC32-K00270_0050_AHL2FHBBXX_ATCACG_S5_...,b0a7cdf2-2ad8-4442-91b0-548ea4975554,Tumor,170818_UNC32-K00270_0050_AHL2FHBBXX_ATCACG_S5_...,c83987a5-1c13-4af4-b46c-218fe5f60c34,NaN,NaN,YES,PASS
4,S005,C3L-00090,No,3,12,129C,C3L-00090-01,CPT0001140003,Tumor,No,...,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_GAGTGG_S10...,8ce5618d-9ff6-40f9-aeea-8d8e1633ae38,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_GAGTGG_S10...,06d3fd4a-a623-4146-8500-4f1f17235253,Tumor,a6524c2d-d7dd-4629-980e-b45dbdc92c49,YES,PASS


Our goal is to harmonize the data from our source table (`dou.csv`) with the data from our target table `cao.csv`

In [3]:
df_target = pd.read_csv("./datasets/Dou-ucec-confirmatory.csv")
df_target.head(5)

,Idx,Case_id,Case_excluded,Batch,Plex,ReporterName,Aliquot_ID,Group,Discovery_study,Age,...,Follow-up_additional_surgery_for_new_tumor,Follow-up_additional_treatment_radiation_therapy_for_new_tumor,Follow-up_additional_treatment_pharmaceutical_therapy_for_new_tumor,Follow-up_additional_treatment_immuno_for_new_tumor,Follow-up_days_from_date_of_collection_to_date_of_last_contact,Follow-up_cause_of_death,Follow-up_days_from_date_of_initial_pathologic_diagnosis_to_date_of_death,Follow-up_days_from_date_of_initial_pathologic_diagnosis_to_date_of_additional_surgery_for_new_tumor,Follow-up_procedure_type_of_new_tumor,Follow-up_residual_tumor_after_surgery_for_new_tumor
0,C3L-00086,C3L-00086,No,b4,16.0,128N,CPT0092460003,Tumor,No,56,...,n/a|No|No|No|No,n/a|Yes|Yes|Yes|Yes,n/a|Yes|Yes|Yes|Yes,n/a|No|No|No|No,330.0|701.0|1046.0|1436.0|n/a,n/a|n/a|n/a|n/a|Breast Carcinoma,n/a|n/a|n/a|n/a|1578.0,n/a|n/a|n/a|n/a|n/a,n/a|n/a|n/a|n/a|n/a,n/a|n/a|n/a|n/a|n/a
1,C3L-00898,C3L-00898,No,b4,14.0,128C,CPT0172200008,Tumor,No,54,...,n/a|n/a|n/a|n/a,n/a|n/a|n/a|n/a,n/a|n/a|n/a|n/a,n/a|n/a|n/a|n/a,396.0|746.0|982.0|1600.0,n/a|n/a|n/a|n/a,n/a|n/a|n/a|n/a,n/a|n/a|n/a|n/a,n/a|n/a|n/a|n/a,n/a|n/a|n/a|n/a
2,C3L-00943,C3L-00943,No,b4,15.0,130C,CPT0086090003,Tumor,No,63,...,n/a|n/a|n/a,n/a|n/a|n/a,n/a|n/a|n/a,n/a|n/a|n/a,237.0|693.0|1039.0,n/a|n/a|n/a,n/a|n/a|n/a,n/a|n/a|n/a,n/a|n/a|n/a,n/a|n/a|n/a
3,C3L-01064,C3L-01064,No,b3,9.0,129N,CPT0113430004,Tumor,No,54,...,No|No|No|No,No|Yes|No|No,Yes|Yes|Yes|Yes,No|No|No|No,453.0|726.0|1062.0|1447.0,n/a|n/a|n/a|n/a,n/a|n/a|n/a|n/a,n/a|n/a|n/a|n/a,n/a|n/a|n/a|n/a,n/a|n/a|n/a|n/a
4,C3L-01277,C3L-01277,No,b4,13.0,130N,CPT0093170003,Tumor,No,61,...,n/a|No|No,n/a|No|Yes,n/a|Yes|No,n/a|No|No,351.0|713.0|967.0,n/a|n/a|n/a,n/a|n/a|n/a,n/a|n/a|n/a,n/a|n/a|n/a,n/a|n/a|n/a


For this demo, we are going to focus on a subset of the columns.

In [4]:
column_names = [
    "Proteomics_Participant_ID",
    "Case_excluded",
    "Proteomics_TMT_channel",
    "Proteomics_Aliquot_ID",
    "Proteomics_Tumor_Normal",
    "Country",
    "Histologic_type",
    "Path_Stage_Primary_Tumor-pT",
    "Path_Stage_Reg_Lymph_Nodes-pN",
    "Clin_Stage_Dist_Mets-cM",
    "tumor_Stage-Pathological",
    "BMI",
    "Age",
    "Diabetes",
    "Race",
    "Ethnicity",
    "Gender",
    "Tumor_Site",
    "Tumor_Site_Other",
    "Tumor_Focality",
    "Tumor_Size_cm",
    "Estrogen_Receptor",
    "Progesterone_Receptor",
    "MLH1",
    "PMS2",
    "p53",
    "MLH1_Promoter_Hypermethylation",
    "Num_full_term_pregnancies",
    "ESTIMATE_StromalScore",
    "ESTIMATE_ESTIMATEScore",
    "CNV_class",
    "MSI_status",
    "POLE_subtype",
    "Genomics_subtype",
]
df_source = df_source[column_names]
df_source.head(5)

,Proteomics_Participant_ID,Case_excluded,Proteomics_TMT_channel,Proteomics_Aliquot_ID,Proteomics_Tumor_Normal,Country,Histologic_type,Path_Stage_Primary_Tumor-pT,Path_Stage_Reg_Lymph_Nodes-pN,Clin_Stage_Dist_Mets-cM,...,PMS2,p53,MLH1_Promoter_Hypermethylation,Num_full_term_pregnancies,ESTIMATE_StromalScore,ESTIMATE_ESTIMATEScore,CNV_class,MSI_status,POLE_subtype,Genomics_subtype
0,C3L-00006,No,128N,CPT0001460012,Tumor,United States,Endometrioid,pT1a (FIGO IA),pN0,cM0,...,Intact nuclear expression,Cannot be determined,Cannot be determined,1,2821.088429,7706.697310,CNV_LOW,MSI-H,No,MSI-H
1,C3L-00008,No,130N,CPT0001300009,Tumor,United States,Endometrioid,pT1a (FIGO IA),pNX,cM0,...,Loss of nuclear expression,Cannot be determined,Cannot be determined,1,2714.248104,6346.448091,CNV_LOW,MSI-H,No,MSI-H
2,C3L-00032,No,131,CPT0001420009,Tumor,United States,Endometrioid,pT1a (FIGO IA),pN0,cM0,...,Intact nuclear expression,Cannot be determined,Cannot be determined,4 or more,3856.501277,10459.413600,CNV_LOW,MSS,No,CNV_low
3,C3L-00084,Yes,129N,CPT0000820012,Tumor,NaN,Carcinosarcoma,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSS,No,NaN
4,C3L-00090,No,129C,CPT0001140003,Tumor,United States,Endometrioid,pT1a (FIGO IA),pNX,cM0,...,Intact nuclear expression,Cannot be determined,Cannot be determined,4 or more,3012.556593,7475.466867,CNV_LOW,MSS,No,CNV_low


#### Finding column matches between two tables

`bdi-kit` offers a suite of functions to help with data harmonization tasks.

For instance, it can help automatically discover one-to-one mappings between the source and target dataset columns.

To do so using `bdi-kit`, we can use the `match_schema()` function to match columns of the two schemas as follows. The `method` parameter allows modifying the schema matching algorithm used to perform the matching.

```python
bdi.match_schema(df_source, df_target, method="jaccard_distance")
bdi.match_schema(df_source, df_target, method="coma")
bdi.match_schema(df_source, df_target, method="similarity_flooding")
```

The method we use below is based on a pre-trained language model fine-tuned for the biomedical domain.

For all methods available, see: [https://bdi-kit.readthedocs.io/en/latest/schema-matching.html](https://bdi-kit.readthedocs.io/en/latest/schema-matching.html)


In [5]:
schema_mapping = bdi.match_schema(df_source, df_target, method="max_val_sim")
schema_mapping.head(15)

Extracting features from 34 columns...


  0%|          | 0/34 [00:00<?, ?it/s]

Extracting features from 213 columns...


  0%|          | 0/213 [00:00<?, ?it/s]

,source,target
0,Proteomics_Participant_ID,Case_id
1,Case_excluded,Case_excluded
2,Proteomics_TMT_channel,ReporterName
3,Proteomics_Aliquot_ID,Aliquot_ID
4,Proteomics_Tumor_Normal,Group
5,Country,Participant_country
6,Histologic_type,Histologic_Type
7,Path_Stage_Primary_Tumor-pT,Pathologic_staging_primary_tumor_pt
8,Path_Stage_Reg_Lymph_Nodes-pN,Pathologic_staging_regional_lymph_nodes_pn
9,Clin_Stage_Dist_Mets-cM,Clinical_staging_distant_metastasis_cm


### Inspecting values of each column

In [6]:
bdi.preview_domain(df_source, 'Proteomics_Participant_ID').head(5)

,value_name
0,C3L-00006
1,C3L-00008
2,C3L-00032
3,C3L-00084
4,C3L-00090


In [7]:
bdi.preview_domain(df_target, 'Case_id').head(5)

,value_name
0,C3L-00086
1,C3L-00898
2,C3L-00943
3,C3L-01064
4,C3L-01277


#### Finding value matches between two columns

Once the matching columns are identified, we can standardize data to ensure that no duplicate values represent the same entity/meaning.

To do that, `bdikit` provides the function `match_values()` to find values that should potentially be merged. The library supports multiple methods to perform this task, including syntactic and semantic matching algorithms. In this example, we use the `tfidf` method, which finds values based on the similarity of character n-grams. Please, refer to the [bdikit documentation](https://bdi-kit.readthedocs.io/) to learn more about the methods available.

In [8]:
# Matches values from each pair of source-target columns
value_matches = bdi.match_values(df_source, df_target, schema_mapping, method="tfidf")

# Print value matches
bdi.view_value_matches(value_matches, edit=False)

<br>**Source column:** Case_excluded<br>**Target column:** Case_excluded<br>

,source,target,similarity
0,No,No,1.0
1,Yes,Yes,1.0


<br>**Source column:** Proteomics_TMT_channel<br>**Target column:** ReporterName<br>

,source,target,similarity
0,128N,128N,1.000
1,130N,130N,1.000
2,129N,129N,1.000
3,129C,129C,1.000
4,130C,130C,1.000
5,127N,127N,1.000
6,127C,127C,1.000
7,128C,128C,1.000
8,131,131N,0.714


<br>**Source column:** Proteomics_Tumor_Normal<br>**Target column:** Group<br>

...

,source,target,similarity
0,MSI-H,MSI-H,1.0
1,MSS,MSS,1.0
2,NaN,NaN,1.0


<br>**Source column:** POLE_subtype<br>**Target column:** POLE<br>

,source,target,similarity
0,No,No,1.0
1,Yes,Yes,1.0
2,NaN,NaN,1.0


<br>**Source column:** Genomics_subtype<br>**Target column:** Genomic_subtype<br>

,source,target,similarity
0,MSI-H,MSI-H,1.000
1,NaN,NaN,1.000
2,POLE,POLE,1.000
3,CNV_low,CNV_L,0.663
4,CNV_high,CNV_H,0.632


#### Generating a harmonized table

After verifying and fixing value mappings, we can proceed to generate a harmonized table.

In bdikit, we can do this using the `bdi.materialize_mapping()` function.

In [9]:
df_mapped = bdi.materialize_mapping(df_source, value_matches)
df_mapped.head(5)

,Case_excluded,ReporterName,Group,Participant_country,Histologic_Type,Pathologic_staging_primary_tumor_pt,Pathologic_staging_regional_lymph_nodes_pn,Clinical_staging_distant_metastasis_cm,Tumor_stage_pathological,Diabetes,...,Ancillary_studies_progesterone_receptor,Ancillary_studies_mlh1,Ancillary_studies_pms2,Ancillary_studies_p53,Ancillary_studies_mlh1_promoter_hypermethylation,Donor_information_number_of_full_term_pregnancies,CNV_status,MSI_status,POLE,Genomic_subtype
0,No,128N,Tumor,United States,Endometrioid carcinoma,pT1a (FIGO IA),pN0,cM0,Stage I,NaN,...,Cannot be determined,Intact nuclear expression,Intact nuclear expression,Cannot be determined,Cannot be determined,1,CNV_L,MSI-H,No,MSI-H
1,No,130N,Tumor,United States,Endometrioid carcinoma,pT1a (FIGO IA),pNX,cM0,Stage IV,NaN,...,Cannot be determined,Intact nuclear expression,Loss of nuclear expression,Cannot be determined,Cannot be determined,1,CNV_L,MSI-H,No,MSI-H
2,No,131N,Tumor,United States,Endometrioid carcinoma,pT1a (FIGO IA),pN0,cM0,Stage I,Yes,...,Cannot be determined,Intact nuclear expression,Intact nuclear expression,Cannot be determined,Cannot be determined,4 or more,CNV_L,MSS,No,CNV_L
3,Yes,129N,Tumor,NaN,Serous carcinoma,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSS,No,NaN
4,No,129C,Tumor,United States,Endometrioid carcinoma,pT1a (FIGO IA),pNX,cM0,Stage I,NaN,...,Cannot be determined,Intact nuclear expression,Intact nuclear expression,Cannot be determined,Cannot be determined,4 or more,CNV_L,MSS,No,CNV_L


For comparisson sake, here is how these columns looked like in the original table:

In [10]:
source_column_names = list(map(lambda m: m.attrs['source'], value_matches))
target_column_names = list(map(lambda m: m.attrs['target'], value_matches))
df_source[source_column_names].head(5)

,Case_excluded,Proteomics_TMT_channel,Proteomics_Tumor_Normal,Country,Histologic_type,Path_Stage_Primary_Tumor-pT,Path_Stage_Reg_Lymph_Nodes-pN,Clin_Stage_Dist_Mets-cM,tumor_Stage-Pathological,Diabetes,...,Progesterone_Receptor,MLH1,PMS2,p53,MLH1_Promoter_Hypermethylation,Num_full_term_pregnancies,CNV_class,MSI_status,POLE_subtype,Genomics_subtype
0,No,128N,Tumor,United States,Endometrioid,pT1a (FIGO IA),pN0,cM0,Stage I,No,...,Cannot be determined,Intact nuclear expression,Intact nuclear expression,Cannot be determined,Cannot be determined,1,CNV_LOW,MSI-H,No,MSI-H
1,No,130N,Tumor,United States,Endometrioid,pT1a (FIGO IA),pNX,cM0,Stage IV,No,...,Cannot be determined,Intact nuclear expression,Loss of nuclear expression,Cannot be determined,Cannot be determined,1,CNV_LOW,MSI-H,No,MSI-H
2,No,131,Tumor,United States,Endometrioid,pT1a (FIGO IA),pN0,cM0,Stage I,Yes,...,Cannot be determined,Intact nuclear expression,Intact nuclear expression,Cannot be determined,Cannot be determined,4 or more,CNV_LOW,MSS,No,CNV_low
3,Yes,129N,Tumor,NaN,Carcinosarcoma,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSS,No,NaN
4,No,129C,Tumor,United States,Endometrioid,pT1a (FIGO IA),pNX,cM0,Stage I,No,...,Cannot be determined,Intact nuclear expression,Intact nuclear expression,Cannot be determined,Cannot be determined,4 or more,CNV_LOW,MSS,No,CNV_low


The final step is to merge the two tables in a single harmonized dataset.

In [11]:
pd.concat([df_mapped[target_column_names], df_target[target_column_names]])

,Case_excluded,ReporterName,Group,Participant_country,Histologic_Type,Pathologic_staging_primary_tumor_pt,Pathologic_staging_regional_lymph_nodes_pn,Clinical_staging_distant_metastasis_cm,Tumor_stage_pathological,Diabetes,...,Ancillary_studies_progesterone_receptor,Ancillary_studies_mlh1,Ancillary_studies_pms2,Ancillary_studies_p53,Ancillary_studies_mlh1_promoter_hypermethylation,Donor_information_number_of_full_term_pregnancies,CNV_status,MSI_status,POLE,Genomic_subtype
0,No,128N,Tumor,United States,Endometrioid carcinoma,pT1a (FIGO IA),pN0,cM0,Stage I,NaN,...,Cannot be determined,Intact nuclear expression,Intact nuclear expression,Cannot be determined,Cannot be determined,1,CNV_L,MSI-H,No,MSI-H
1,No,130N,Tumor,United States,Endometrioid carcinoma,pT1a (FIGO IA),pNX,cM0,Stage IV,NaN,...,Cannot be determined,Intact nuclear expression,Loss of nuclear expression,Cannot be determined,Cannot be determined,1,CNV_L,MSI-H,No,MSI-H
2,No,131N,Tumor,United States,Endometrioid carcinoma,pT1a (FIGO IA),pN0,cM0,Stage I,Yes,...,Cannot be determined,Intact nuclear expression,Intact nuclear expression,Cannot be determined,Cannot be determined,4 or more,CNV_L,MSS,No,CNV_L
3,Yes,129N,Tumor,NaN,Serous carcinoma,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSS,No,NaN
4,No,129C,Tumor,United States,Endometrioid carcinoma,pT1a (FIGO IA),pNX,cM0,Stage I,NaN,...,Cannot be determined,Intact nuclear expression,Intact nuclear expression,Cannot be determined,Cannot be determined,4 or more,CNV_L,MSS,No,CNV_L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,No,129N,Enriched_Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186,No,129C,Enriched_Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,No,129N,Enriched_Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,No,130C,Enriched_Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
